In [0]:
%pip install pandas
%pip install openpyxl
%pip install rich

In [0]:
import pandas as pd

# Pandas is suitable when dealing with medium-size files
df_pandas = pd.read_excel("/Workspace/Users/moraesit@outlook.com/CapstoneProject/gold/preco-gasoleo-gasolina95.xlsx",
                          engine='openpyxl', # When working with .xlsx files
                          sheet_name='gasoleo_gasolina95_semanal',
                          header = 0)
df_gasoleo_gasolina = spark.createDataFrame(df_pandas)

df_gasoleo_gasolina.display()

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_gold.gasoleo_gasolina95_semanal (
    data DATE NOT NULL,
    --Up to 3 digits before the decimal, and 3 after
    PST_euro DECIMAL(6, 3),
    ISP_Out_euro DECIMAL(6, 3),
    IVA_euro DECIMAL(6, 3),
    PVP_euro DECIMAL(6, 3),
    type VARCHAR(10)
);

In [0]:
from pyspark.sql.functions import expr

df_gasoleo_gasolina_fixed = df_gasoleo_gasolina \
    .withColumn("PST_euro", expr("try_cast(PST_euro as DECIMAL(6,3))")) \
    .withColumn("ISP_Out_euro", expr("try_cast(ISP_Out_euro as DECIMAL(6,3))")) \
    .withColumn("IVA_euro", expr("try_cast(IVA_euro as DECIMAL(6,3))")) \
    .withColumn("PVP_euro", expr("try_cast(PVP_euro as DECIMAL(6,3))")) \

df_gasoleo_gasolina_fixed.display()

In [0]:
#keeps the table definition intact, only replacing the rows.
df_gasoleo_gasolina_fixed.write.mode("overwrite") \
                               .insertInto("workspace.sc_gold.gasoleo_gasolina95_semanal")


In [0]:
%sql
SELECT *
FROM workspace.sc_gold.gasoleo_gasolina95_semanal

In [0]:
import pandas as pd

# Pandas is suitable when dealing with medium-size files
pandas_energia = pd.read_excel("/Workspace/Users/moraesit@outlook.com/CapstoneProject/gold/preco-energia.xlsx",
                                  engine='openpyxl', # When working with .xlsx files
                                  sheet_name='energia_diario_hora',
                                  header = 0)
df_energia = spark.createDataFrame(pandas_energia)

df_energia.display()

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.sc_gold.energia_diario_hora (
    data DATE NOT NULL,
    hora TINYINT NOT NULL,
    --Up to 3 digits before the decimal, and 2 after
    euro_per_megawatt DECIMAL(5, 2)
);

In [0]:
#keeps the table definition intact, only replacing the rows.
df_energia.write.mode("overwrite") \
                .insertInto("workspace.sc_gold.energia_diario_hora")

In [0]:
%sql
SELECT *
FROM workspace.sc_gold.energia_diario_hora
ORDER BY data, hora

In [0]:
%sql
SELECT count(*)
FROM workspace.sc_gold.energia_diario_hora